In [1]:
#from keras_transformer import get_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import Adam
from TransformerEncoder import TransformerEncoder
from TrainablePositionalEmbeddings import TransformerPositionalEmbedding
from keras.layers import Input,Dense,Embedding,Lambda,TimeDistributed,LSTM, Reshape, Dropout
from keras.models import Model

Using TensorFlow backend.


## data

In [70]:
sku_id = 30851
n_in = 16
#data load
df = pd.read_csv('data/sku_day_20191023_v2.csv')
data_beer = df[df["sku_id"] == sku_id].copy()
data_beer['time'] = pd.to_datetime(data_beer['time'], format='%Y-%m')
data_beer['week'] = ((data_beer['time'] - pd.datetime(year=2014, month=1, day=6)).dt.days / 7).astype(int)
data_beer['monthly'] = data_beer['time'].apply(lambda x: pd.datetime(year=x.year, month=x.month, day=1))
data_beer_week = data_beer.groupby(['week'])['y'].sum().to_frame().sort_values(['week']).reset_index()
data_beer_week = pd.DataFrame(data_beer_week.y.values).values

#data perpare
#n_in = n_seq * n_steps
n_out = 1
df = pd.DataFrame(data_beer_week)
cols = list()
for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
for i in range(0, n_out):
    cols.append(df.shift(-i))
agg = pd.concat(cols, axis=1)
agg.dropna(inplace=True)
data_supervised = agg.values

In [71]:
mu = pd.Series(data_supervised.reshape(-1,)).mean()
sigma = pd.Series(data_supervised.reshape(-1,)).std()

In [74]:
data = data_supervised

In [29]:
data_supervised[data_supervised<mu-sigma]=0
data_supervised[(data_supervised>=mu-sigma) & (data_supervised<mu)]=1
data_supervised[(data_supervised>=mu) & (data_supervised<mu + sigma)]=2
data_supervised[(data_supervised>=mu+sigma) & (data_supervised<mu + 2*sigma)]=3
data_supervised[data_supervised>=mu+2*sigma]=4

In [32]:
data = data_supervised

In [75]:
data_train, data_test = train_test_split(data, test_size=0.05, random_state=42)
#data_train, data_test = data

In [34]:
x_train = data_train[:,:-1]
y_train = to_categorical(data_train[:,-1]) # one-hot
x_test = data_test[:,:-1]
y_test = to_categorical(data_test[:,-1]) # one-hot

## Model

In [41]:
#hyper parameter
input_seq_len = 16
input_dim = 1
hidden_dim = 30
positional_ff_dim = 32

In [49]:
def build_model():
    '''
    embedding: uniform initialization
    encoder + softmax
    '''
    encoder_inputs = Input((16,),name='Encoder_input')
    next_step_input = Embedding(5,hidden_dim,input_length=16)(encoder_inputs)
    next_step_input = Dropout(0.2)(next_step_input)


    positional_embedding_layer    = TransformerPositionalEmbedding(name='Positional_embedding')
    next_step_input               = positional_embedding_layer(next_step_input)

    next_step_input,attention =TransformerEncoder(hidden_dim,
                                                             1,
                                                             hidden_dim,
                                                             hidden_dim,
                                                             positional_ff_dim,
                                                             dropout_rate= 0.1,
                                                             name= 'Transformer')(next_step_input)
    '''
    next_step_input,attention =TransformerEncoder(hidden_dim,
                                                         1,
                                                         hidden_dim,
                                                         hidden_dim,
                                                         positional_ff_dim,
                                                         dropout_rate= 0.1,
                                                         name= 'Transformer1')(next_step_input)
    
    '''
    next_step_input = Lambda(lambda x: x[:,0])(next_step_input)
    next_step_input = Dropout(0.5)(next_step_input)
    outputs = Dense(5, activation='softmax')(next_step_input)
    model = Model(encoder_inputs, outputs)
    model.summary()
    
    return model

In [50]:
model = build_model()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_input (InputLayer)      (None, 16)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 16, 30)       150         Encoder_input[0][0]              
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 16, 30)       0           embedding_3[0][0]                
__________________________________________________________________________________________________
Positional_embedding (Transform (None, 16, 30)       480         dropout_9[0][0]                  
____________________________________________________________________________________________

In [51]:
model.compile(optimizer=Adam(),loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
history = model.fit(x_train, y_train, epochs=300)

Epoch 1/200
203/203 [==============================] - 0s 203us/step - loss: 1.0620 - accuracy: 0.6256
Epoch 2/200
203/203 [==============================] - 0s 122us/step - loss: 1.0517 - accuracy: 0.5862
Epoch 3/200
203/203 [==============================] - 0s 137us/step - loss: 1.0187 - accuracy: 0.6355
Epoch 4/200
203/203 [==============================] - 0s 126us/step - loss: 1.0028 - accuracy: 0.6108
Epoch 5/200
203/203 [==============================] - 0s 132us/step - loss: 1.0360 - accuracy: 0.6059
Epoch 6/200
203/203 [==============================] - 0s 129us/step - loss: 1.0439 - accuracy: 0.6010
Epoch 7/200
203/203 [==============================] - 0s 126us/step - loss: 1.0641 - accuracy: 0.5961
Epoch 8/200
203/203 [==============================] - 0s 124us/step - loss: 0.9658 - accuracy: 0.6502
Epoch 9/200
203/203 [==============================] - 0s 126us/step - loss: 1.0236 - accuracy: 0.6355
Epoch 10/200
203/203 [==============================] - 0s 119us/step - l

203/203 [==============================] - 0s 115us/step - loss: 0.7979 - accuracy: 0.7044
Epoch 158/200
203/203 [==============================] - 0s 113us/step - loss: 0.7143 - accuracy: 0.6946
Epoch 159/200
203/203 [==============================] - 0s 115us/step - loss: 0.8075 - accuracy: 0.6601
Epoch 160/200
203/203 [==============================] - 0s 113us/step - loss: 0.7433 - accuracy: 0.7340
Epoch 161/200
203/203 [==============================] - 0s 114us/step - loss: 0.7484 - accuracy: 0.7044
Epoch 162/200
203/203 [==============================] - 0s 113us/step - loss: 0.7309 - accuracy: 0.7192
Epoch 163/200
203/203 [==============================] - 0s 114us/step - loss: 0.7924 - accuracy: 0.6847
Epoch 164/200
203/203 [==============================] - 0s 113us/step - loss: 0.7981 - accuracy: 0.6700
Epoch 165/200
203/203 [==============================] - 0s 111us/step - loss: 0.7128 - accuracy: 0.7192
Epoch 166/200
203/203 [==============================] - 0s 114us/ste

## Validation

In [55]:
model.evaluate(x_test[:8,:],y_test[:8,:])

11/11 [==============================] - 0s 204us/step


[1.5636924505233765, 0.5454545617103577]

In [57]:
preds = model.predict(x_test)
'''
for i in range(preds.shape[0]):
    for j in range(preds.shape[1]):
        preds[i,j] = round(preds[i,j],2)
'''
preds = np.round(preds, 2)

In [58]:
print('break point: ', 0,round(mu - sigma,2), round(mu,2), round(mu+sigma,2), round(mu+2*sigma,2))

break point:  0 145.96 3451.46 6756.95 10062.45


In [59]:
print(preds) # Probability distribution

[[0.   0.9  0.09 0.01 0.  ]
 [0.05 0.94 0.01 0.   0.  ]
 [0.01 0.98 0.   0.   0.  ]
 [0.01 0.86 0.12 0.   0.  ]
 [0.   0.29 0.13 0.55 0.03]
 [0.04 0.76 0.2  0.   0.  ]
 [0.   0.92 0.07 0.   0.  ]
 [0.01 0.66 0.04 0.27 0.03]
 [0.06 0.63 0.3  0.01 0.  ]
 [0.   0.03 0.22 0.63 0.12]
 [0.02 0.89 0.09 0.01 0.  ]]


In [60]:
preds = model.predict(x_test)
preds = np.argmax(preds, axis=1)
preds # categorical output

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1])

In [61]:
y_test = np.argmax(y_test, axis=1)
y_test # true category

array([2, 0, 2, 2, 3, 1, 1, 1, 1, 4, 1])

In [76]:
data_test[:,-1] # true volume

array([ 6609.6 ,   116.64,  4633.2 ,  3499.2 ,  7795.44,  1581.12,
         952.56,  3402.  ,   220.32, 13186.8 ,   758.16])

In [85]:
from sklearn.metrics import accuracy_score, classification_report

In [88]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.57      0.80      0.67         5
           2       1.00      0.33      0.50         3
           3       0.33      1.00      0.50         1
           4       0.00      0.00      0.00         1

    accuracy                           0.55        11
   macro avg       0.38      0.43      0.33        11
weighted avg       0.56      0.55      0.48        11

